# Лабораторная работа 1

| Студент       | Ланин Олег                    | 
|-----------|------------------------------------|
| Группа  | М8О-301Б-19  |

### Задание

1) Реализовать следующие алгоритмы машинного обучения: Linear/ Logistic Regression, SVM, KNN, Naive Bayes в отдельных классах 
2)  Данные классы должны наследоваться от BaseEstimator и  ClassifierMixin, иметь методы fit и predict (подробнее: https://scikit-learn.org/stable/developers/develop.html )
3) Вы должны организовать весь процесс предобработки, обучения и тестирования с помощью Pipeline (подробнее: https://scikit-learn.org/stable/modules/compose.html)
4) Вы должны настроить гиперпараметры моделей с помощью кросс валидации (GridSearchCV,RandomSearchCV, подробнее здесь: https://scikit-learn.org/stable/modules/grid_search.html), вывести и сохранить эти гиперпараметры в файл, вместе с обученными моделями
5) Проделать аналогично с коробочными решениями
6) Для каждой модели получить оценки метрик:Confusion Matrix,  Accuracy, Recall, Precision, ROC_AUC curve (подробнее: Hands on machine learning with python and scikit learn chapter 3, mlcourse.ai, https://ml-handbook.ru/chapters/model_evaluation/intro)
7) Проанализировать полученные результаты и сделать выводы о применимости моделей
8) Загрузить полученные гиперпараметры модели и обученные модели в формате pickle  на гит вместе с jupyter notebook ваших экспериментов

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RandomizedSearchCV

In [23]:
for df in [df_train, df_test]:
    mean = df['Age'].mean()
    std = df['Age'].std()
    number_of_nulls = df['Age'].isnull().sum()
    random_ages = np.random.randint(mean - std, mean + std, size=number_of_nulls)

    new_ages = df['Age'].copy()
    new_ages[np.isnan(new_ages)] = random_ages
    df['Age'] = new_ages

for df in [df_train, df_test]:
    df['Embarked'] = df['Embarked'].fillna('S')

df_test = df_test[df_test['Fare'].notnull()]

df_train = df_train.drop(columns=['Name', 'PassengerId', 'Ticket', 'Cabin'])
df_test = df_test.drop(columns=['Name', 'PassengerId', 'Ticket', 'Cabin'])

df_train['Relatives'] = df_train['Parch'] + df_train['SibSp']
df_test['Relatives'] = df_test['Parch'] + df_test['SibSp']
df_train = df_train.drop(columns=['SibSp', 'Parch'])
df_test = df_test.drop(columns=['SibSp', 'Parch'])

genders = {'male': 0, 'female': 1}
ports = {"S": 0, "C": 1, "Q": 2}
for df in [df_train, df_test]:
    df['Sex'] = df['Sex'].map(genders)
    df['Embarked'] = df['Embarked'].map(ports)

In [24]:
X_train = df_train.drop(columns=['Survived']).to_numpy()
Y_train = df_train['Survived'].to_numpy()

X_test = df_test.drop(columns=['Survived']).to_numpy()
Y_test = df_test['Survived'].to_numpy()

df_test.head()

,Pclass,Sex,Age,Fare,Embarked,Survived,Relatives
0,3,0,34.5,7.8292,2,0,0
1,3,1,47.0,7.0000,0,1,1
2,2,0,62.0,9.6875,2,0,0
3,3,0,27.0,8.6625,0,0,0
4,3,1,22.0,12.2875,0,1,2


Вспомогательные функции для анализа результатов классификации

In [25]:
def confusion_matrix(y_pred, y_test):
    matrix = pd.DataFrame({'actual_1' : [0, 0], 'actual_0': [0, 0]})
    matrix.index = ['predicted_1', 'predicted_0']

    for i in range(len(y_pred)):
        if y_pred[i] == 1 and y_test[i] == 1:
            matrix.loc['predicted_1', 'actual_1'] += 1
        elif y_pred[i] == 1 and y_test[i] == 0:
            matrix.loc['predicted_1', 'actual_0'] += 1
        elif y_pred[i] == 0 and y_test[i] == 1:
            matrix.loc['predicted_0', 'actual_1'] += 1
        else:
            matrix.loc['predicted_0', 'actual_0'] += 1

    return matrix

def metrics(matrix):
    TP = matrix.loc['predicted_1', 'actual_1']
    FP = matrix.loc['predicted_1', 'actual_0']
    FN = matrix.loc['predicted_0', 'actual_1']
    TN = matrix.loc['predicted_0', 'actual_0']

    accuracy = (TP + TN) / (TP + TN + FP + FN)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)

    return accuracy, precision, recall

# Построение моделей
## 1. KNN

In [260]:
class WeightedKNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, K=3):
        self.K = K
    
    def fit(self, X_train, Y_train):
        self.X_train = X_train.to_numpy()
        self.Y_train = Y_train.to_numpy()

    def predict(self, X_test):
        X_test = X_test.to_numpy()

        # Нормализуем данные в каждом столбце
        self.X_train = np.apply_along_axis(lambda x: (x-x.mean())/ x.std(), 0, self.X_train)
        X_test = np.apply_along_axis(lambda x: (x-x.mean())/ x.std(), 0, X_test)

        Y_pred = []
        
        for p in X_test:        
            distances = list()
            # Считаем расстояния от данной точки до всех остальных
            for row_idx, row in enumerate(self.X_train):
                distance = 0
                for feature_idx in range(len(p)):
                    distance += (p[feature_idx] - row[feature_idx])**2

                # Добавляем в список пару расстояние-класс
                distances.append([math.sqrt(distance), Y_train[row_idx]])

            # Находим К ближайших точек
            k_closest_points = sorted(distances, key=lambda x: x[0])[:self.K]

            # Находим числа, обратные к каждому расстоянию
            inverse_distances = list()
            for dist in k_closest_points:
                inverse_distances.append(1/dist[0])
            
            # Находим сумму этих обратных чисел
            sum_of_inverses = sum(inverse_distances)
            
            # Находим вес для каждой точки + соответствующий ей класс
            weights = [[inverse/sum_of_inverses, k_closest_points[idx][1]] for idx, inverse in enumerate(inverse_distances)]

            # Считаем вероятности для каждого класса
            probabilities = {c : 0 for c in Y_train.unique()}
            for elem in weights:
                probabilities[elem[1]] += elem[0]
            
            # Возвращаем класс с максимальной вероятностью
            Y_pred.append(max(probabilities, key=probabilities.get))

        return np.array(Y_pred)

    def accuracy_score(self, Y_test, Y_pred):
	    return sum(np.array(Y_pred) == np.array(Y_test)) / len(Y_test)


In [261]:
'''
for i in range(2, 15):
    my_knn = WeightedKNNClassifier(K=i)
    my_knn.fit(X_train, Y_train)
    Y_pred = my_knn.predict(X_test)
    print(f"Accuracy for weighted KNN with K = {i} : {my_knn.accuracy_score(Y_test, Y_pred)}")
'''
# Из данного теста мы выяснили, что оптимальный K = 7

my_knn = WeightedKNNClassifier(K=7)
my_knn.fit(X_train, Y_train)
Y_pred = my_knn.predict(X_test)
print(f"Accuracy for weighted KNN with K = 7 : {my_knn.accuracy_score(Y_test, Y_pred)}")

Accuracy for weighted KNN with K = 7 : 0.8800959232613909


In [262]:
scores = []
for i in range(2, 100):
    knn = KNeighborsClassifier(n_neighbors = i)
    knn.fit(X_train, Y_train)
    Y_pred = knn.predict(X_test)
    scores.append((Y_test == Y_pred).sum() / len(Y_test))

print(f'Best accuracy for sklearn\'s KNN: {max(scores)}')

Best accuracy for sklearn's KNN: 0.6714628297362111


In [263]:
# Same with normalized data

X_train_normalized = X_train.apply(lambda x: (x-x.mean())/ x.std(), axis=0)
X_test_normalized = X_test.apply(lambda x: (x-x.mean())/ x.std(), axis=0)

scores = []
for i in range(2, 100):
    knn = KNeighborsClassifier(n_neighbors = i)
    knn.fit(X_train_normalized, Y_train)
    Y_pred = knn.predict(X_test_normalized)
    scores.append((Y_test == Y_pred).sum() / len(Y_test))

print(f'Best accuracy for sklearn\'s KNN with normalized data: {max(scores)}')

Best accuracy for sklearn's KNN with normalized data: 0.9760191846522782


## 2. Naive Bayes

In [264]:
class NaiveBayesClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self):
        pass

    def get_probabilities(self, class_idx: int, x: np.array) -> np.array:
        # Считаем вероятность как значение функции плотности нормального распределения в точке x=(p_1,..,p_n)  
    
        # Массив матожиданий и дисперсий для каждого признака при данном классе
        mean = self.mean_cond_class[class_idx]
        var = self.var_cond_class[class_idx]

        # Считаем вероятность для каждого признака, используя плотность вероятности нормального распределения
        exponent = np.exp((-1/2) * ((x-mean)**2) / (2 * var))
        probabilities = exponent / np.sqrt(2 * np.pi * var)
        return probabilities

    # Найдем P(class|p_1,..,p_n)
    def get_posterior(self, x: np.array) -> int:
        posteriors = []
        for class_idx in range(self.num_of_classes):
            prior = np.log(self.prior[class_idx])
            # Условная вероятность получить такие значения признаков для этого класса
            conditional = np.sum(np.log(self.get_probabilities(class_idx, x)))
            posterior = prior + conditional
            posteriors.append(posterior)
        # Возвращаем класс, для которого такая вероятность максимальна
        return self.classes[np.argmax(posteriors)]

    def fit(self, X_train, Y_train):
        self.classes = np.unique(Y_test)
        self.num_of_classes = len(self.classes)
        
        # Посчитаем выборочные средние и дисперсии для каждого признака в зависимости от класса
        self.mean_cond_class = X_train.groupby(Y_train).apply(np.mean).to_numpy()
        self.var_cond_class = X_train.groupby(Y_train).apply(np.var).to_numpy()

        # Считаем для каждого класса, сколько наблюдений принадлежит этому классу
        self.prior = X_train.groupby(Y_train).apply(lambda col: len(col))
        # Находим оценку вероятности того, что случайное наблюдение принадлежит этому классу
        # путём деления результатов на общее количество наблюдений
        self.prior = np.array(self.prior / len(Y_train))
        
    def predict(self, X_test):
        Y_pred = [self.get_posterior(f) for f in X_test.to_numpy()]
        return Y_pred
    
    def accuracy_score(self, Y_test, Y_pred):
        return sum(Y_pred == Y_test) / len(Y_test)

In [265]:
nbc = NaiveBayesClassifier()
nbc.fit(X_train, Y_train)
Y_pred = nbc.predict(X_test)
print(f'Accuracy of custom Naive Bayes: {nbc.accuracy_score(Y_test, Y_pred)}')

Accuracy of custom Naive Bayes: 0.8033573141486811


In [266]:
gaussian = GaussianNB() 
gaussian.fit(X_train, Y_train) 
Y_pred = gaussian.predict(X_test)  
print(f'Accuracy of sklearn Naive Bayes: {gaussian.score(X_train, Y_train)}')

Accuracy of sklearn Naive Bayes: 0.797979797979798


## 3. Логистическая регрессия

In [26]:
class my_Logistic_Regression(BaseEstimator, ClassifierMixin):
    def __init__(self, learning_rate, n_iterations):        
        self.learning_rate = learning_rate        
        self.n_iterations = n_iterations

    def fit(self, X, Y):
        self.X = X.copy()
        self.Y = Y.copy()
        n_samples, n_features = X.shape     
        self.W = np.zeros(n_features)        
        self.b = 0

        # обновление весов с помощью градиентного спуска
        for _ in range(self.n_iterations):
            linear_model = np.dot(self.X, self.W) + self.b          
            y_predicted = 1 / (1 + np.exp(-linear_model))

            dW = (1 / n_samples) * np.dot(X.T, y_predicted - Y)
            db = (1 / n_samples) * np.sum(y_predicted - Y)
            self.W -= self.learning_rate * dW    
            self.b -= self.learning_rate * db  

    def predict(self, X):
        S = np.array(1 / (1 + np.exp(-(np.dot(X, self.W) + self.b))))
        # если сигмойд получается больше, чем 0.5, то предсказываем класс 1
        return np.where(S >= 0.5, 1, 0)

In [27]:
hyperparameters = [
(0.1, 500), 
(0.1, 1000), 
(0.1, 1500),
(0.01, 1000),
(0.01, 2000),
(0.01, 5000),
(0.01, 10000),
(0.001, 1000),
(0.001, 3000),
(0.001, 5000),
(0.001, 10000),
(0.001, 15000),
(0.0001, 2000),
(0.0001, 5000),
(0.0001, 10000),
(0.0001, 15000),
(0.0001, 20000),
(0.0001, 50000)
]

for pair in hyperparameters:
    lr, iters = pair
    my_log_regr = my_Logistic_Regression(learning_rate=lr, n_iterations=iters)
    my_log_regr.fit(X_train, Y_train)
    Y_pred_test = my_log_regr.predict(X_test)
    Y_pred_train = my_log_regr.predict(X_train)
    print(f'Learning rate = {lr}, iterations = {iters}, train accuracy = {np.mean(Y_train == Y_pred_train)}, test accuracy = {np.mean(Y_test == Y_pred_test)}')


Learning rate = 0.1, iterations = 500, train accuracy = 0.691358024691358, test accuracy = 0.6690647482014388
Learning rate = 0.1, iterations = 1000, train accuracy = 0.6621773288439955, test accuracy = 0.657074340527578
Learning rate = 0.1, iterations = 1500, train accuracy = 0.6161616161616161, test accuracy = 0.6354916067146283
Learning rate = 0.01, iterations = 1000, train accuracy = 0.6352413019079686, test accuracy = 0.657074340527578
Learning rate = 0.01, iterations = 2000, train accuracy = 0.6531986531986532, test accuracy = 0.6618705035971223
Learning rate = 0.01, iterations = 5000, train accuracy = 0.7216610549943884, test accuracy = 0.762589928057554
Learning rate = 0.01, iterations = 10000, train accuracy = 0.7530864197530864, test accuracy = 0.8465227817745803
Learning rate = 0.001, iterations = 1000, train accuracy = 0.6879910213243546, test accuracy = 0.6498800959232613
Learning rate = 0.001, iterations = 3000, train accuracy = 0.7014590347923682, test accuracy = 0.67386

In [28]:
mylogit = my_Logistic_Regression(learning_rate=0.001, n_iterations=15000)
mylogit.fit(X_train, Y_train)
Y_pred = mylogit.predict(X_test)

In [29]:
confusion_matrix(Y_pred, Y_test)

,actual_1,actual_0
predicted_1,103,16
predicted_0,49,249


In [30]:
metrics_mylogreg = metrics(confusion_matrix(Y_pred, Y_test))
print(f'Accuracy: {metrics_mylogreg[0]}\nPrecision: {metrics_mylogreg[1]}\nRecall: {metrics_mylogreg[2]}')

Accuracy: 0.8441247002398081
Precision: 0.865546218487395
Recall: 0.6776315789473685


Мы получили довольно низкий recall. Это значит (и видно из таблицы), что модель предсказала малую долю выживших людей, то есть много выживших она определила как погибших.

Теперь посмотрим на модель из sklearn

In [31]:
logreg_model = LogisticRegression().fit(X_train, Y_train)
Y_pred = logreg_model.predict(X_test)

In [32]:
confusion_matrix(Y_pred, Y_test)

,actual_1,actual_0
predicted_1,140,16
predicted_0,12,249


In [33]:
metrics_sklogreg = metrics(confusion_matrix(Y_pred, Y_test))
print(f'Accuracy: {metrics_sklogreg[0]}\nPrecision: {metrics_sklogreg[1]}\nRecall: {metrics_sklogreg[2]}')

Accuracy: 0.9328537170263789
Precision: 0.8974358974358975
Recall: 0.9210526315789473


Мы видим, что модель логистический регрессии из sklearn справилась немного лучше, чем реализованная. Причем recall получился немного выше, чем precision.

## 4. SVM

In [34]:
class my_SVM(BaseEstimator, ClassifierMixin):
    def __init__(self, learning_rate=0.01, lambda_param=0.01, n_iters=10000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters

    def fit(self, X, y):
        # переименовываем лейблы в -1 и 1
        y_ = np.where(y <= 0, -1, 1)
        n_samples, n_features = X.shape

        self.W = np.zeros(n_features)
        self.b = 0

        # процесс обучения (настройка весов и смещения)
        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y_[idx] * ((x_i @ self.W) - self.b) >= 1
                if condition:
                    self.W -= self.lr * (2 * self.lambda_param * self.W)
                else:
                    self.W -= self.lr * (2 * self.lambda_param * self.W - np.dot(x_i,y_[idx]))
                    self.b -= self.lr * y_[idx]

    def predict(self, X):
        res = np.dot(X, self.W) - self.b
        # предсказываем класс в зависимости от знака
        return np.where(res >= 0, 1, 0)

In [35]:
hyperparameters = [
(0.01, 0.01, 2000),
(0.01, 0.01, 5000),
(0.001, 0.01, 1000),
(0.001, 0.01, 3000),
(0.001, 0.01, 5000),
(0.001, 0.01, 10000),
(0.001, 0.01, 15000),
(0.0001, 0.01, 5000),
(0.0001, 0.01, 10000),
(0.0001, 0.01, 15000),
]

for h in hyperparameters:
    lr, lambda_param, iters = h
    my_svm = my_SVM(learning_rate=lr, lambda_param=lambda_param, n_iters=iters)
    my_svm.fit(X_train, Y_train)
    print(f'Learning rate = {lr}, lambda = {lambda_param}, iterations = {iters}, train accuracy = {np.mean(Y_train == my_svm.predict(X_train))}, test accuracy = {np.mean(Y_test == my_svm.predict(X_test))}')


Learning rate = 0.01, lambda = 0.01, iterations = 2000, train accuracy = 0.7441077441077442, test accuracy = 0.7146282973621103
Learning rate = 0.01, lambda = 0.01, iterations = 5000, train accuracy = 0.7227833894500562, test accuracy = 0.7122302158273381
Learning rate = 0.001, lambda = 0.01, iterations = 1000, train accuracy = 0.7777777777777778, test accuracy = 0.8537170263788969
Learning rate = 0.001, lambda = 0.01, iterations = 3000, train accuracy = 0.7710437710437711, test accuracy = 0.8513189448441247
Learning rate = 0.001, lambda = 0.01, iterations = 5000, train accuracy = 0.7912457912457912, test accuracy = 0.8441247002398081
Learning rate = 0.001, lambda = 0.01, iterations = 10000, train accuracy = 0.7822671156004489, test accuracy = 0.86810551558753
Learning rate = 0.001, lambda = 0.01, iterations = 15000, train accuracy = 0.7699214365881033, test accuracy = 0.8441247002398081
Learning rate = 0.0001, lambda = 0.01, iterations = 5000, train accuracy = 0.7946127946127947, test

Рассмотрим модель с параметрами (0.0001, 0.01, 5000).

In [36]:
my_svm = my_SVM(learning_rate=0.0001, lambda_param=0.01, n_iters=5000)
my_svm.fit(X_train, Y_train)
Y_pred = my_svm.predict(X_test)

In [37]:
confusion_matrix(Y_pred, Y_test)

,actual_1,actual_0
predicted_1,138,10
predicted_0,14,255


In [38]:
metrics_my_svm = metrics(confusion_matrix(Y_pred, Y_test))
print(f'Accuracy: {metrics_my_svm[0]}\nPrecision: {metrics_my_svm[1]}\nRecall: {metrics_my_svm[2]}')

Accuracy: 0.9424460431654677
Precision: 0.9324324324324325
Recall: 0.9078947368421053


Посмотрим на модель SVM из sklearn

In [40]:
sklearn_svm_model = SVC()
sklearn_svm_model.fit(X_train, Y_train)
Y_pred = sklearn_svm_model.predict(X_test)

In [41]:
confusion_matrix(Y_pred, Y_test)

,actual_1,actual_0
predicted_1,41,36
predicted_0,111,229


In [42]:
metrics_sklearn_svm = metrics(confusion_matrix(Y_pred, Y_test))
print(f'Accuracy: {metrics_sklearn_svm[0]}\nPrecision: {metrics_sklearn_svm[1]}\nRecall: {metrics_sklearn_svm[2]}')

Accuracy: 0.6474820143884892
Precision: 0.5324675324675324
Recall: 0.26973684210526316


In [43]:
# С нормализацией данных по столбцам
X_train_normalized = np.apply_along_axis(lambda x: (x-x.mean())/ x.std(), 0, X_train)
X_test_normalized = np.apply_along_axis(lambda x: (x-x.mean())/ x.std(), 0, X_test)

sklearn_svm_model = SVC()
sklearn_svm_model.fit(X_train_normalized, Y_train)
Y_pred = sklearn_svm_model.predict(X_test_normalized)
confusion_matrix(Y_pred, Y_test)

,actual_1,actual_0
predicted_1,113,8
predicted_0,39,257


In [44]:
metrics_sklearn_svm = metrics(confusion_matrix(Y_pred, Y_test))
print(f'Accuracy: {metrics_sklearn_svm[0]}\nPrecision: {metrics_sklearn_svm[1]}\nRecall: {metrics_sklearn_svm[2]}')

Accuracy: 0.8872901678657075
Precision: 0.9338842975206612
Recall: 0.743421052631579


Мы видим, что нормализация данных сильно улучшила качество классификации. Подберем гиперпараметры с помощью RandomizedSearchCV.

In [45]:
def svc_param_selection(X, y):
    Cs = [0.001, 0.01, 0.1, 1, 10, 100]
    gammas = [0.001, 0.01, 0.1, 1]
    kernels = ['linear', 'rbf']
    param_grid = {'C': Cs, 'gamma' : gammas, 'kernel': kernels}
    search = RandomizedSearchCV(SVC(), param_grid)
    search.fit(X, y)
    search.best_params_
    return search.best_params_

svc_param_selection(X_train_normalized, Y_train)

{'kernel': 'rbf', 'gamma': 0.1, 'C': 1}

In [46]:
sklearn_svm_model = SVC(kernel='rbf', gamma=0.01, C=100)
sklearn_svm_model.fit(X_train_normalized, Y_train)
Y_pred = sklearn_svm_model.predict(X_test_normalized)

In [47]:
confusion_matrix(Y_pred, Y_test)

,actual_1,actual_0
predicted_1,134,3
predicted_0,18,262


In [48]:
metrics_sklearn_svm = metrics(confusion_matrix(Y_pred, Y_test))
print(f'Accuracy: {metrics_sklearn_svm[0]}\nPrecision: {metrics_sklearn_svm[1]}\nRecall: {metrics_sklearn_svm[2]}')

Accuracy: 0.9496402877697842
Precision: 0.9781021897810219
Recall: 0.881578947368421


Выполняемой мною лабораторной работы мной были изучены и составлены 4 предложенных классификатора. Logistic Regression, SVM, KNN, Naive Bayes. С помощью их удалось добиться хороших результатов. Все результаты больше 85%. Во время выполнения работы также использовал метод GridSearchCV, для определения лучших параметров и использовании кросс-валидации. В результате чего, удалось добиться еще более лучших результатов даже 95%.

Для меня задание оказалось очень сложным.